In [ ]:
# install additional dependencies 
!pip install spacy_langdetect
!pip install contractions
!pip install pycountry
!pip install textblob
!pip install openpyxl

!pip install keras_preprocessing
!pip install keras
!pip install tensorflow
!pip install sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import re

from bs4 import BeautifulSoup
import pandas_profiling as pp

#Importing libraries for text pre-processing
import spacy
from spacy_langdetect import LanguageDetector
import nltk
nltk.download('stopwords')
from nltk.tokenize.toktok import ToktokTokenizer
#from nltk.tokenize import word_tokenize
from contractions import contractions_dict
import unicodedata
from typing import Dict, List

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split 


from keras.layers import Dense, Embedding, LSTM, GRU, Dropout, Bidirectional, TimeDistributed
from keras.layers import Activation, Concatenate, SpatialDropout1D, Input, Lambda, Flatten
from keras.callbacks import EarlyStopping 

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, Flatten, Reshape
from keras.layers import Concatenate, concatenate

from keras.callbacks import ModelCheckpoint
import keras



[nltk_data] Downloading package stopwords to /Users/vinay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
env = "local" # colab
project_path_local = "../data/"
project_path_colab = "/content/drive/My Drive/AIML 2019 GreatLearning/Capstone Project NLP/POC"
project_path = project_path_local if env == "local" else project_path_colab
print(f"project_path: {project_path}")

project_path: ../data/


In [8]:
def combine_target_variables(row, count=3, original_target_col="Assignment group", count_col="tmp_target_count"):
        if row[count_col] <= count:
            return "OTHER"
        else:
            return row[original_target_col]

In [11]:
df_en_orig = pd.read_excel(f"{project_path}/Input Data Synthetic CleanedV3.xlsx")

In [12]:
df_en = df_en_orig[df_en_orig['lang_textblob']=='en']
df_en.reset_index(inplace=True)
df_en["tmp_target_count"] = df_en.groupby(["Assignment group"])["Description"].transform("count") 
for index, row in df_en.iterrows():
        df_en.loc[index, "target1"] = combine_target_variables(row)


df_en.shape

(5291, 14)

In [25]:
print("df_en['target1']\n", df_en['target1'].value_counts())

df_en['target1']
 GRP_0     2511
GRP_8      374
GRP_2      205
GRP_12     180
GRP_19     170
GRP_3      159
GRP_13     122
GRP_14     101
GRP_4       91
GRP_25      87
GRP_5       87
GRP_29      81
GRP_9       79
GRP_16      75
GRP_10      73
GRP_18      71
GRP_7       64
GRP_6       62
GRP_26      49
GRP_34      40
GRP_40      38
GRP_15      36
OTHER       35
GRP_41      35
GRP_28      31
GRP_20      31
GRP_22      29
GRP_11      27
GRP_24      27
GRP_33      26
GRP_21      26
GRP_31      24
GRP_45      23
GRP_23      20
GRP_1       20
GRP_39      15
GRP_60      14
GRP_37      14
GRP_47      13
GRP_44      12
GRP_27      12
GRP_50      10
GRP_62      10
GRP_30      10
GRP_17       9
GRP_36       9
GRP_53       8
GRP_65       8
GRP_51       7
GRP_52       6
GRP_55       6
GRP_42       5
GRP_59       5
GRP_43       5
GRP_46       4
Name: target1, dtype: int64


In [13]:
### Paramters
df_en['description_cleaned_wc'] = df_en['description_cleaned'].apply(lambda x: len(str(x).split(" ")))
df_en['short_description_cleaned_wc'] = df_en['short_description_cleaned'].apply(lambda x: len(str(x).split(" ")))
MAX_WORDS = int(np.percentile(df_en['description_cleaned_wc'], 95))  ## based on 95 percentile
SHORT_DESC_MAX_WORDS = int(np.percentile(df_en['short_description_cleaned_wc'], 95))  ## based on 95 percentile
VALIDATION_SPLIT = 0.2 
print(f"MAX_WORDS: {MAX_WORDS}")
print(f"SHORT_DESC_MAX_WORDS: {SHORT_DESC_MAX_WORDS}")


MAX_WORDS: 85
SHORT_DESC_MAX_WORDS: 11


In [14]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(df_en.description_cleaned)
tokenizer.fit_on_texts(df_en.short_description_cleaned.apply(lambda x: f"{x}"))

word_counts = tokenizer.word_counts
word_docs = tokenizer.word_docs
word_index = tokenizer.word_index
index_word = tokenizer.index_word
document_count = tokenizer.document_count

In [15]:
no_of_descriptions = df_en.description_cleaned.size
data_shape = (no_of_descriptions, MAX_WORDS)
data = np.zeros(data_shape, dtype=np.int64)
print("data.shape", data.shape)


for description_i, description in enumerate(df_en.description_cleaned.to_list()):
    # print(f"{description_i+1} of {no_of_descriptions}")
    for word_i, word in enumerate(text_to_word_sequence(description)):
        encoded_word = word_index[word]
        if word_i >= MAX_WORDS:
            break
        elif word_i < MAX_WORDS:
            # attempt to update data only if 
            # sentence_i < MAX_SENTS and word_i < MAX_SENT_LENGTH
            data[description_i][word_i] = encoded_word

print("data[:2,:]", data[:2,:])


data.shape (5291, 85)
data[:2,:] [[ 406    6  119  142   90   73   52    6   73   18    5  189    6   33
    52  295  163   53   33    9  116    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [  66  296  104  296    1  297   31  544 3476  189  261    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]]


In [16]:
no_of_short_descriptions = df_en.short_description_cleaned.size
short_data_shape = (no_of_short_descriptions, SHORT_DESC_MAX_WORDS)
short_data = np.zeros(short_data_shape, dtype=np.int64)
print("short_data.shape", short_data.shape)


for description_i, description in enumerate(df_en.short_description_cleaned.apply(lambda x: f"{x}").to_list()):
    # print(f"{description_i+1} of {no_of_short_descriptions}")
    for word_i, word in enumerate(text_to_word_sequence(description)):
        encoded_word = word_index[word]
        if word_i >= SHORT_DESC_MAX_WORDS:
            break
        elif word_i < SHORT_DESC_MAX_WORDS:
            # attempt to update short_data only if 
            # sentence_i < MAX_SENTS and word_i < MAX_SENT_LENGTH
            short_data[description_i][word_i] = encoded_word

print("short_data[:2,:]", short_data[:2,:])
print(index_word[33], index_word[9])
print(index_word[31])

short_data.shape (5291, 11)
short_data[:2,:] [[33  9  0  0  0  0  0  0  0  0  0]
 [31  0  0  0  0  0  0  0  0  0  0]]
login issue
outlook


In [17]:
# getting unique labels in given data
labels = pd.get_dummies(df_en['target1']).values 

TARGET_LEN = len(df_en.target1.unique())
print(f"TARGET_LEN: {TARGET_LEN}")
print(df_en.target1.unique())

TARGET_LEN: 55
['GRP_0' 'GRP_1' 'GRP_3' 'GRP_4' 'GRP_5' 'GRP_8' 'GRP_6' 'GRP_10' 'GRP_9'
 'GRP_11' 'GRP_14' 'GRP_15' 'GRP_17' 'GRP_18' 'GRP_2' 'GRP_19' 'GRP_20'
 'GRP_21' 'GRP_25' 'GRP_13' 'GRP_16' 'GRP_26' 'GRP_27' 'GRP_28' 'GRP_29'
 'GRP_30' 'GRP_31' 'GRP_22' 'GRP_24' 'GRP_7' 'GRP_12' 'GRP_34' 'OTHER'
 'GRP_36' 'GRP_37' 'GRP_33' 'GRP_39' 'GRP_40' 'GRP_41' 'GRP_43' 'GRP_44'
 'GRP_45' 'GRP_46' 'GRP_47' 'GRP_23' 'GRP_50' 'GRP_42' 'GRP_51' 'GRP_52'
 'GRP_53' 'GRP_55' 'GRP_59' 'GRP_60' 'GRP_62' 'GRP_65']


In [18]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = VALIDATION_SPLIT, random_state=9) 
x_train_short, x_test_short, y_train_short, y_test_short = train_test_split(short_data, labels, test_size = VALIDATION_SPLIT, random_state=9) 
print("x_train.shape: ", x_train.shape, "  y_train.shape:", y_train.shape)
print("x_test.shape: ", x_test.shape, "  y_test.shape: ", y_test.shape) 

print("x_train_short.shape: ", x_train_short.shape, "  y_train_short.shape:", y_train_short.shape)
print("x_test_short.shape: ", x_test_short.shape, "  y_test_short.shape: ", y_test_short.shape) 

x_train.shape:  (4232, 85)   y_train.shape: (4232, 55)
x_test.shape:  (1059, 85)   y_test.shape:  (1059, 55)
x_train_short.shape:  (4232, 11)   y_train_short.shape: (4232, 55)
x_test_short.shape:  (1059, 11)   y_test_short.shape:  (1059, 55)


In [26]:
### trying with regularizing embedding
OUTPUT_LEN = 150 # embedding_dimentations
vocab_size = len(tokenizer.word_index.keys()) + 1

description_encoder_inputs = Input(shape=(MAX_WORDS,))

# x1 = Reshape((400,))(description_encoder_inputs)
x2 = Embedding(
        output_dim=OUTPUT_LEN, 
        input_dim=vocab_size, 
        input_length=MAX_WORDS, 
        embeddings_regularizer=keras.regularizers.l2(.001))(description_encoder_inputs)

description_encoder = LSTM(OUTPUT_LEN, dropout=0.25, recurrent_dropout=0.25) 
description_encoder_outputs = description_encoder(x2) # description_state_h, description_state_c


short_description_encoder_inputs = Input(shape=(SHORT_DESC_MAX_WORDS,))

# sx1 = Reshape((400,))(short_description_encoder_inputs)
sx2 = Embedding(
        output_dim=OUTPUT_LEN, 
        input_dim=vocab_size, 
        input_length=SHORT_DESC_MAX_WORDS, 
        embeddings_regularizer=keras.regularizers.l2(.001))(short_description_encoder_inputs)

short_description_encoder = LSTM(OUTPUT_LEN, dropout=0.25, recurrent_dropout=0.25) 
short_description_encoder_outputs = short_description_encoder(sx2) # description_state_h, description_state_c

combined_context_vector = concatenate([description_encoder_outputs, short_description_encoder_outputs])

d1 = Dense(units = 256, activation='relu')(combined_context_vector) # combined_context_vector
d2 = Dropout(0.2)(d1)
predictions = Dense(TARGET_LEN, activation='softmax')(d2) 

LSTM_model = Model(inputs=[description_encoder_inputs, short_description_encoder_inputs], outputs=predictions)
LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


print(LSTM_model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 85)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 85, 150)      1440150     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 11, 150)      1440150     input_4[0][0]                    
____________________________________________________________________________________________

In [27]:
%%time
# batch_size = 32
# model.fit(x_train_re, y_train, epochs=5, batch_size=batch_size, verbose=2, validation_split=VALIDATION_SPLIT)

batch_size = 64
epochs = 10
# filepath = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
# filepath = "saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath = "saved_models/lstm-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='auto')

print('Training...')

LSTM_model.fit([x_train, x_train_short], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=VALIDATION_SPLIT,
          callbacks=[checkpoint]) 

Training...
Train on 3385 samples, validate on 847 samples
Epoch 1/10
3385/3385 [==============================] - 35s 10ms/step - loss: 3.2937 - accuracy: 0.4815 - val_loss: 2.2783 - val_accuracy: 0.5148

Epoch 00001: saving model to saved_models/lstm-01-0.51.hdf5
Epoch 2/10
3385/3385 [==============================] - 33s 10ms/step - loss: 2.1126 - accuracy: 0.5448 - val_loss: 2.2013 - val_accuracy: 0.5242

Epoch 00002: saving model to saved_models/lstm-02-0.52.hdf5
Epoch 3/10
3385/3385 [==============================] - 33s 10ms/step - loss: 1.9719 - accuracy: 0.5501 - val_loss: 2.1296 - val_accuracy: 0.5289

Epoch 00003: saving model to saved_models/lstm-03-0.53.hdf5
Epoch 4/10
3385/3385 [==============================] - 29s 9ms/step - loss: 1.8945 - accuracy: 0.5613 - val_loss: 2.1475 - val_accuracy: 0.5218

Epoch 00004: saving model to saved_models/lstm-04-0.52.hdf5
Epoch 5/10
3385/3385 [==============================] - 34s 10ms/step - loss: 1.8660 - accuracy: 0.5634 - val_loss

In [28]:
%%time
# evaluating model on validation data set
 
loss, acc = LSTM_model.evaluate([x_test, x_test_short], y_test, batch_size=batch_size)
print('Test loss:', loss)
print('Test accuracy:', acc)

1059/1059 [==============================] - 3s 3ms/step
Test loss: 2.154598119463754
Test accuracy: 0.5514636635780334
CPU times: user 4.75 s, sys: 720 ms, total: 5.47 s
Wall time: 2.67 s


In [29]:
y_pred = LSTM_model.predict([x_test, x_test_short], verbose=1)
y_pred_index = np.argmax(y_pred, axis=1)
y_pred_class = (y_pred == y_pred.max(axis=1, keepdims=True)).astype(int)
print(f"y_pred_class: {y_pred_class[1]}")



1059/1059 [==============================] - 3s 3ms/step
y_pred_class: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [30]:
from sklearn.metrics import classification_report
print("y_pred_class: ", y_pred_class[:1])
print("y_test: ", y_test[:1])
print(classification_report(y_test, y_pred_class, target_names=df_en.target1.unique()))

y_pred_class:  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
y_test:  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
              precision    recall  f1-score   support

       GRP_0       0.71      0.90      0.79       511
       GRP_1       0.00      0.00      0.00         6
       GRP_3       0.00      0.00      0.00        10
       GRP_4       0.00      0.00      0.00         5
       GRP_5       0.29      0.55      0.38        29
       GRP_8       0.21      0.57      0.31        28
       GRP_6       0.00      0.00      0.00        17
      GRP_10       0.00      0.00      0.00         5
       GRP_9       0.00      0.00      0.00        19
      GRP_11       0.00      0.00      0.00         4
      GRP_14       0.00      0.00      0.00        14
      GRP_15       0.00      0.00      0.00        36
      GRP_17       0.48      0.30      0.